In [ ]:
# !pip install -U qdrant-client fastembed datasets -q

In [27]:
import os
import json
import pandas as pd
from datasets import load_dataset
from qdrant_client import QdrantClient

In [ ]:
dataset = load_dataset("nirantk/ecommerce-icecat-dataset", split="train")
collection_name = 'ecommerce-icecat-dataset'

In [ ]:
qdrant_client = QdrantClient(
    os.environ["QDRANT_API_URL"],
    api_key = os.environ["QDRANT_API_KEY"],
)

In [ ]:
names = [c.name for c in qdrant_client.get_collections().collections]
names

In [ ]:
if collection_name in names:
    qdrant_client.delete_collection(collection_name)

In [ ]:
qdrant_client.get_collections()

In [ ]:
dataset

In [ ]:
def make_document(example):
    return {
        'document': f"Name: {example['name']}\nTitle: {example['title']}\nDescription:{example['short_description']}",
    }

new_dataset = dataset.map(lambda example: make_document(example))

In [ ]:
new_dataset

In [ ]:
sample_dataset = new_dataset.select(range(1000))
text_documents = sample_dataset['document']
ds = sample_dataset.remove_columns(['document', 'id', 'ean', 'price'])
metadata = ds.to_pandas().to_dict(orient='records')

In [ ]:
# metadata

In [19]:
isinstance(metadata, list), isinstance(metadata[0], dict)

(True, True)

In [ ]:
inserted_ids = qdrant_client.add(
    collection_name=collection_name,
    documents= text_documents,
    metadata = metadata
)

print(f"Inserted {len(inserted_ids)} documents")

In [35]:
def query_and_display(query_text: str):
    results = qdrant_client.query(
        collection_name=collection_name,
        query_text=query_text,
        limit=10,
    )
    for idx, result in enumerate(results):
        print(f"---Position: {idx}---")
        print(f"---Score: {result.score}---")
        print(result.document, result.score)

query_and_display("What is a good laptop for gaming?")

---Position: 0---
---Score: 0.87856674---
Name: Compaq nc2400 Notebook PC
Title: HP Compaq nc2400 Notebook PC
Description:Compaq nc2400 Notebook PC 0.87856674
---Position: 1---
---Score: 0.877485---
Name: Compaq 8510p Notebook PC
Title: HP Compaq 8510p Notebook PC
Description:Compaq 8510p Notebook PC 0.877485
---Position: 2---
---Score: 0.87729037---
Name: 255 G7
Title: HP 255 G7 Notebook Black 39.6 cm (15.6") 1920 x 1080 pixels AMD Ryzen 3 8 GB DDR4-SDRAM 256 GB SSD Wi-Fi 5 (802.11ac) Windows 10 Pro
Description:None 0.87729037
---Position: 3---
---Score: 0.87464297---
Name: FY949AA
Title: HP FY949AA graphics card NVIDIA Quadro FX 3800 1 GB GDDR3
Description:NVIDIA Quadro FX3800 1GB PCIe Graphics Card 0.87464297
---Position: 4---
---Score: 0.8735425---
Name: KD060AT
Title: HP KD060AT graphics card GDDR2
Description:ATI Radeon HD 2400XT (256MB DH) PCIe Graphics Card 0.8735425
---Position: 5---
---Score: 0.87204087---
Name: 15 WB-7US1130SH
Title: Gigabyte AERO 15 WB-7US1130SH Notebook Bl

In [36]:
query_and_display("Printer for home use")

---Position: 0---
---Score: 0.91898155---
Name: Color LaserJet 5500n Printer
Title: HP Color LaserJet 5500n Printer
Description:Color LaserJet 5500n Printer 0.91898155
---Position: 1---
---Score: 0.91878814---
Name: LaserJet 3030 all-in-one printer/fax/scanner/copier
Title: HP LaserJet 3030 all-in-one printer/fax/scanner/copier
Description:LaserJet 3030 All-in-One Printer 0.91878814
---Position: 2---
---Score: 0.91801757---
Name: Deskjet 460cb Mobile Printer
Title: HP Deskjet 460cb Mobile Printer inkjet printer Colour 4800 x 1200 DPI A4
Description:Deskjet 460cb 0.91801757
---Position: 3---
---Score: 0.91674614---
Name: DesignJet 50ps Printer
Title: HP DesignJet 50ps Printer
Description:Designjet 50ps Printer 0.91674614
---Position: 4---
---Score: 0.9158038---
Name: A626
Title: HP Photosmart A626 photo printer Inkjet 4800 x 1200 DPI
Description:Photosmart A626 0.9158038
---Position: 5---
---Score: 0.91522676---
Name: DesignJet 500ps Plus 42-in Printer
Title: HP DesignJet 500ps Plus 42-